In [23]:
import sqlite3
import pandas as pd

# Connect to the database
database_path = '../data/database.sqlite' 
connection = sqlite3.connect(database_path)
cursor = connection.cursor()

# Query to list all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all results
tables = cursor.fetchall()

# Print the table names
print("Tables in the database:")
for table in tables:
    print(table[0])  # Each row is a tuple, print the first element

Tables in the database:
Country
CountryNotes
Series
Indicators
SeriesNotes
Footnotes
G7_GDPs


In [24]:
# Query to select the first few rows (e.g., first 5 rows)
query = f"SELECT * FROM Country LIMIT 5;"

# Load the results into a DataFrame
df = pd.read_sql_query(query, connection)
df

,CountryCode,ShortName,TableName,LongName,Alpha2Code,CurrencyUnit,SpecialNotes,Region,IncomeGroup,Wb2Code,...,GovernmentAccountingConcept,ImfDataDisseminationStandard,LatestPopulationCensus,LatestHouseholdSurvey,SourceOfMostRecentIncomeAndExpenditureData,VitalRegistrationComplete,LatestAgriculturalCensus,LatestIndustrialData,LatestTradeData,LatestWaterWithdrawalData
0,AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,Fiscal year end: March 20; reporting period fo...,South Asia,Low income,AF,...,Consolidated central government,General Data Dissemination System (GDDS),1979,"Multiple Indicator Cluster Survey (MICS), 2010/11","Integrated household survey (IHS), 2008",,2013/14,,2013,2000
1,ALB,Albania,Albania,Republic of Albania,AL,Albanian lek,,Europe & Central Asia,Upper middle income,AL,...,Budgetary central government,General Data Dissemination System (GDDS),2011,"Demographic and Health Survey (DHS), 2008/09",Living Standards Measurement Study Survey (LSM...,Yes,2012,2011,2013,2006
2,DZA,Algeria,Algeria,People's Democratic Republic of Algeria,DZ,Algerian dinar,,Middle East & North Africa,Upper middle income,DZ,...,Budgetary central government,General Data Dissemination System (GDDS),2008,"Multiple Indicator Cluster Survey (MICS), 2012","Integrated household survey (IHS), 1995",,,2010,2013,2001
3,ASM,American Samoa,American Samoa,American Samoa,AS,U.S. dollar,,East Asia & Pacific,Upper middle income,AS,...,,,2010,,,Yes,2007,,,
4,ADO,Andorra,Andorra,Principality of Andorra,AD,Euro,,Europe & Central Asia,High income: nonOECD,AD,...,,,2011. Population data compiled from administra...,,,Yes,,,2006,


In [25]:
# Query to select the first few rows (e.g., first 5 rows)
query = f"SELECT * FROM Indicators LIMIT 5;"

# Load the results into a DataFrame
df = pd.read_sql_query(query, connection)
df

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,1.335609e+02
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,8.779760e+01
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,6.634579e+00
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,8.102333e+01
4,Arab World,ARB,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,3.000000e+06


In [26]:
# SQL query to get unique IndicatorName values starting with "GDP"
query = '''
SELECT DISTINCT IndicatorName
FROM Indicators
WHERE IndicatorName LIKE 'GDP%';
'''

# Load the results into a DataFrame
df = pd.read_sql_query(query, connection)
df

,IndicatorName
0,GDP (constant LCU)
1,GDP (current LCU)
2,GDP at market prices (constant 2005 US$)
3,GDP at market prices (current US$)
4,GDP deflator (base year varies by country)
5,GDP growth (annual %)
6,GDP per capita (constant 2005 US$)
7,GDP per capita (constant LCU)
8,GDP per capita (current LCU)
9,GDP per capita (current US$)


In [27]:
# SQL query to get records for specific GDP indicators for G7 countries, China, and India
query = '''
SELECT *
FROM Indicators
WHERE IndicatorName IN ('GDP at market prices (current US$)', 'GDP per capita (current US$)')
AND CountryCode IN ('CAN', 'FRA', 'DEU', 'ITA', 'JPN', 'GBR', 'USA', 'CHN', 'IND');  -- G7 + China and India
'''

# Load the results into a DataFrame
gdp_indicators_df = pd.read_sql_query(query, connection)

In [28]:
gdp_indicators_df["CountryCode"].unique()

array(['CAN', 'CHN', 'FRA', 'IND', 'ITA', 'JPN', 'GBR', 'USA', 'DEU'],
      dtype=object)

In [29]:
# SQL query
drop_statement = "DROP TABLE IF EXISTS G7_GDPs"
connection.execute(drop_statement)

# SQL query to create a new table G7_GDPs and store the result of the query
create_table_query = '''
CREATE TABLE G7_GDPs AS
SELECT *
FROM Indicators
WHERE IndicatorName IN ('GDP at market prices (current US$)', 'GDP per capita (current US$)')
AND CountryCode IN ('CAN', 'FRA', 'DEU', 'ITA', 'JPN', 'GBR', 'USA', 'CHN', 'IND');  -- G7 + China and India
'''

# Execute the query to create the new table
connection.execute(create_table_query)

# Verify that the data is stored in the new table by querying the first few rows
verify_query = '''
SELECT *
FROM G7_GDPs
LIMIT 10;  -- Retrieve the first 10 rows for verification
'''

# Load the results into a DataFrame to check the contents
g7_gdps_df = pd.read_sql_query(verify_query, connection)
g7_gdps_df

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Canada,CAN,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,4.109345e+10
1,China,CHN,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,5.918412e+10
2,France,FRA,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,6.265147e+10
3,India,IND,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,3.767927e+10
4,Italy,ITA,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,4.038529e+10
5,Japan,JPN,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,4.430734e+10
6,United Kingdom,GBR,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,7.232805e+10
7,United States,USA,GDP at market prices (current US$),NY.GDP.MKTP.CD,1960,5.433000e+11
8,Canada,CAN,GDP at market prices (current US$),NY.GDP.MKTP.CD,1961,4.076797e+10
9,China,CHN,GDP at market prices (current US$),NY.GDP.MKTP.CD,1961,4.955705e+10


In [30]:
# Close the cursor and connection
cursor.close()
connection.close()